In [1]:
import backtrader as bt
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf

from backtesting import Backtest, Strategy
from backtesting.lib import crossover

In [11]:
# class VWAP(bt.Indicator):
#     lines = ('vwap',)
#     plotinfo = dict(subplot=False)
    
#     def next(self):
#         # Calculate cumulative sum of price*volume and volume up to the current bar
#         total_pv = sum(self.data.close[i] * self.data.volume[i] for i in range(len(self.data)))
#         total_vol = sum(self.data.volume[i] for i in range(len(self.data)))
#         self.lines.vwap[0] = total_pv / total_vol if total_vol != 0 else self.data.close[0]


class VWAPRollingIndicator(bt.Indicator):
    """
    Volume Weighted Average Price (VWAP) indicator, rolling calculation.
    """

    lines = ("vwap_rolling",)
    params = {"period": 14}
    plotinfo = {"subplot": False}
    plotlines = {"vwap_rolling": {"color": "green"}}

    def __init__(self) -> None:
        self.hlc = (self.data.high + self.data.low + self.data.close) / 3.0
        self.hlc_volume_sum = bt.ind.SumN(self.hlc * self.data.volume, period=self.p.period)
        self.volume_sum = bt.ind.SumN(self.data.volume, period=self.p.period)

        self.lines.vwap_rolling = bt.DivByZero(self.hlc_volume_sum, self.volume_sum, None) 


        
class VWAPStrategy(bt.Strategy):
    params = (
        ("stop_loss", 0.02),  # 2% stop loss
        # ("take_profit", 0.04),  # 4% take profit
        ("risk", 0.01),  # 1% risk
        ("pivot_window", 5),  # 5 bars for pivot high/low
        ("rrr", 2),  # Risk-Reward Ratio
        ("trailpercent", 0.05),     # Percent Price for Trailing stop
        ("sl_pips", 50),
        ("pip_value", 0.01)     # Pip value for Gold (XAUUSD)
    )
    def __init__(self):
        # initialize the VWAP indicator on data feed
        self.vwap = VWAPRollingIndicator(self.data)
        self.crossover = bt.indicators.CrossOver(self.data.close, self.vwap)
        self.order = None
        self.prev_pivot_high = self.data.low[0]
        self.prev_pivot_low = self.data.high[0]
        
    def notify_order(self, order):
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(f'BUY EXECUTED, {order.executed.price:.2f}')
            elif order.issell():
                self.log(f'SELL EXECUTED, {order.executed.price:.2f}')
        self.order = None
    
    def log(self, txt):
        dt = self.data.datetime.date(0)
        print(f'{dt.isoformat()} {txt}')
    
    def _calculate_size(self, price, sl_price):
        # Calculate the size of the position based on the risk percentage and stop loss
        size = (self.broker.getvalue() * self.params.risk) / abs(price - sl_price)
        return size
    
    def _update_pivots(self):
        if np.max(self.data.high.get(size=self.params.pivot_window * 2 + 1)) == self.data.high[self.params.pivot_window]:
            self.prev_pivot_high = self.data.high[self.params.pivot_window]   
        if np.min(self.data.low.get(size=self.params.pivot_window * 2 + 1)) == self.data.low[self.params.pivot_window]:
            self.prev_pivot_low = self.data.low[self.params.pivot_window]      

    def next(self):
        # if not in a market position, check for crossover from below
        if not self.position:
            if self.crossover > 0 and self.data.close[1] > self.data.open[1]:
                price = self.data.close[0]
                # sl_price = self.data.low[1]
                # sl_price = price * (1 - self.params.stop_loss)
                sl_price = price - self.params.sl_pips * self.params.pip_value
                tp_price = price + np.abs(sl_price - price) * self.params.rrr
                if tp_price < price:
                    self.log(f'Take profit price {tp_price:.2f} is less than entry price {price:.2f}, not placing order.')
                    return
                if sl_price > price:
                    self.log(f'Stop loss price {sl_price:.2f} is greater than entry price {price:.2f}, not placing order.')
                    return

                # size = self._calculate_size(price, sl_price)
                self.order = self.buy_bracket(
                    exectype=bt.Order.Market, 
                    # size=size,
                    price=price,
                    stopprice=sl_price,
                    limitprice=tp_price,
                    stopargs=dict(exectype=bt.Order.Stop),
                    limitargs=dict(exectype=bt.Order.Limit),
                    # trailpercent=self.params.trailpercent
                )
                self.log(f"Bracket order placed: Entry {price:.2f}, SL {sl_price:.2f}, TP {tp_price:.2f}")
 
            # if in a position, check for crossover from above for exit
            elif self.crossover < 0 and self.data.close[1] < self.data.open[1]:
                price = self.data.close[0]
                # sl_price = self.data.high[1]
                # sl_price = price * (1 + self.params.stop_loss)
                sl_price = price + self.params.sl_pips * self.params.pip_value
                tp_price = price - np.abs(sl_price - price) * self.params.rrr
                if tp_price > price:
                    self.log(f'Take profit price {tp_price:.2f} is greater than entry price {price:.2f}, not placing order.')
                    return
                if sl_price < price:
                    self.log(f'Stop loss price {sl_price:.2f} is less than entry price {price:.2f}, not placing order.')
                    return
                # size = self._calculate_size(price, sl_price)
                self.order = self.sell_bracket(
                    exectype=bt.Order.Market, 
                    # size=size,
                    price=price,
                    stopprice=sl_price,
                    limitprice=tp_price,
                    stopargs=dict(exectype=bt.Order.Stop),
                    limitargs=dict(exectype=bt.Order.Limit),
                    # trailpercent=self.params.trailpercent
                )
                self.log(f"Bracket order placed: Entry {price:.2f}, SL {sl_price:.2f}, TP {tp_price:.2f}")
        else:
            pass
            


## Download Data from YF

In [12]:
# Download SPX500 data from Yahoo Finance
yf_params = {'start': '2018-01-01', 'end': '2024-01-01', 'interval': '1d', 'multi_level_index': False}
# data = yf.download(tickers="^GSPC", **yf_params)
# data = yf.download(tickers="DJIA", **yf_params)
# data = yf.download(tickers="^VIX", **yf_params)
# data = yf.download(tickers="^RUT", **yf_params)
# data = yf.download(tickers="^IXIC", **yf_params)
# data = yf.download(tickers="GOOG", **yf_params)
# data = yf.download(tickers="AAPL", **yf_params)
# data = yf.download(tickers="MSFT", **yf_params)
# data = yf.download(tickers="AMZN", **yf_params)
# data = yf.download(tickers="TSLA", **yf_params)
# data = yf.download(tickers="META", **yf_params)
data = yf.download(tickers="NFLX", **yf_params)
# data = yf.download(tickers="NVDA", **yf_params)
# data = yf.download(tickers="AMD", **yf_params)
data.reset_index(inplace=True)
data['Date'] = pd.to_datetime(data['Date'])
# data.set_index('Date', inplace=True)

# data

[*********************100%***********************]  1 of 1 completed


## Source data locally

In [13]:
data_local = pd.read_csv("./data/xau_usd_5m.csv", index_col=None)
data_local['datetime'] = pd.to_datetime(data_local['datetime'])
print(data_local.head())

data_feed_local = bt.feeds.PandasData(dataname=data_local, name="xau",
                                      datetime=0, openinterest=-1,
                                      open=1,
                                      high=2,
                                      low=3,
                                      close=4,
                                      volume=5)

             datetime         open         high          low        close  \
0 2020-02-02 23:00:00  1591.197998  1591.197998  1587.751953  1588.687012   
1 2020-02-02 23:05:00  1588.734009  1590.536987  1588.064941  1590.447998   
2 2020-02-02 23:10:00  1590.500000  1592.167969  1588.849976  1589.462036   
3 2020-02-02 23:15:00  1589.421997  1590.144043  1588.848999  1589.894043   
4 2020-02-02 23:20:00  1589.943970  1590.021973  1588.385010  1588.942017   

   volume  
0     547  
1     372  
2     606  
3     242  
4     246  


In [ ]:
cerebro = bt.Cerebro()
cerebro.addstrategy(VWAPStrategy, 
                    # stop_loss=0.10, 
                    # trailpercent=0.05, 
                    # pivot_window=5, 
                    sl_pips=300,
                    pip_value=0.01,
                    rrr=2.0,
                    risk=0.02
                    )
cerebro.broker.setcash(100_000.0)
cerebro.broker.setcommission(commission=0.001)
data_feed = bt.feeds.pandafeed.PandasData(
    dataname=data,
    datetime=0,
    openinterest=-1,
    open=1,
    high=2,
    low=3,
    close=4,
    volume=5,
)
cerebro.adddata(data_feed_local)

In [24]:
cerebro.addsizer(bt.sizers.PercentSizer, percents=0.02)
cerebro.addanalyzer(bt.analyzers.SharpeRatio, riskfreerate=0.01)
cerebro.addanalyzer(bt.analyzers.DrawDown)
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer)
cerebro.addanalyzer(bt.analyzers.Returns)

results = cerebro.run()

2020-02-03 Bracket order placed: Entry 1589.06, SL 1594.06, TP 1579.06
2020-02-03 SELL EXECUTED, 1589.10
2020-02-03 BUY EXECUTED, 1579.06
2020-02-03 Bracket order placed: Entry 1577.94, SL 1572.94, TP 1587.94
2020-02-03 BUY EXECUTED, 1577.90
2020-02-03 SELL EXECUTED, 1572.94
2020-02-03 Bracket order placed: Entry 1574.99, SL 1569.99, TP 1584.99
2020-02-03 BUY EXECUTED, 1575.03
2020-02-04 SELL EXECUTED, 1569.99
2020-02-04 Bracket order placed: Entry 1567.46, SL 1562.46, TP 1577.46
2020-02-04 BUY EXECUTED, 1567.42
2020-02-04 SELL EXECUTED, 1562.46
2020-02-04 Bracket order placed: Entry 1555.47, SL 1550.47, TP 1565.47
2020-02-04 BUY EXECUTED, 1555.51
2020-02-04 SELL EXECUTED, 1550.47
2020-02-04 Bracket order placed: Entry 1551.77, SL 1546.77, TP 1561.77
2020-02-04 BUY EXECUTED, 1551.73
2020-02-05 SELL EXECUTED, 1561.77
2020-02-05 Bracket order placed: Entry 1561.12, SL 1556.12, TP 1571.12
2020-02-05 BUY EXECUTED, 1561.16
2020-02-05 SELL EXECUTED, 1556.12
2020-02-05 Bracket order placed: E

In [25]:
strat = results[0]
sharpe_ratio = strat.analyzers.sharperatio.get_analysis()
drawdown = strat.analyzers.drawdown.get_analysis()
trade_analysis = strat.analyzers.tradeanalyzer.get_analysis()
returns = strat.analyzers.returns.get_analysis()
print(f'Sharpe Ratio: {sharpe_ratio}')
print(f'Max Drawdown: {drawdown["max"]["drawdown"]}')
print(f'Number of Trades: {trade_analysis.total.closed}')
print(f'Winning Trades: {trade_analysis.won.total}')
print(f'Losing Trades: {trade_analysis.lost.total}')
print(f'Average Trade Return: {returns["rnorm"]}')

Sharpe Ratio: OrderedDict({'sharperatio': -122.31271677207444})
Max Drawdown: 0.1499822792125559
Number of Trades: 4892
Winning Trades: 1949
Losing Trades: 2943
Average Trade Return: -0.00023427156234810372


In [ ]:
fig = cerebro.plot(iplot=True)
plt.show()
fig